In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Define the poisson_prob_above_threshold function
def poisson_prob_above_threshold(mean, threshold):
    from scipy.stats import poisson
    return 1 - poisson.cdf(threshold, mean)

# Sample DataFrame dictionary for demonstration
dfs = {
    '2020': pd.DataFrame({'pts': [10, 20, 15, 30, 25], 'date': pd.date_range(start='1/1/2020', periods=5)}),
    '2021': pd.DataFrame({'pts': [5, 10, 15, 20, 25], 'date': pd.date_range(start='1/1/2021', periods=5)})
}

# Define the threshold
threshold = 18

# Initialize a list to collect results
results = []

# Process each DataFrame in the dictionary
for season, df in dfs.items():
    df = df[['pts', 'date']]
    df = df.assign(rolling_mean=df['pts'].rolling(window=25).mean())
    df = df.assign(prob_above_threshold=df['rolling_mean'].apply(lambda x: poisson_prob_above_threshold(x, 18) if not np.isnan(x) else np.nan))
    df['prob_above_threshold'] = df['prob_above_threshold'].shift(1)
    df = df.dropna()
    df['positive_pred'] = np.where(df['prob_above_threshold'] > 0.5, 1, 0)
    df['positive_true'] = np.where(df['pts'] > threshold, 1, 0)
    dfs[season] = df  # Update the dictionary with the modified DataFrame

    # Calculate evaluation metrics
    y_true = df['positive_true']
    y_pred = df['positive_pred']

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Append results to the list
    results.append({
        'season': season,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': conf_matrix.tolist()  # Convert numpy array to list for display
    })

# Convert results to DataFrame for display
results_df = pd.DataFrame(results)

c:\Users\rusta\.virtualenvs\nba_01-7yhASJUA\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\rusta\.virtualenvs\nba_01-7yhASJUA\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\rusta\.virtualenvs\nba_01-7yhASJUA\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rusta\.virtualenvs\nba_01-7yhASJUA\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(re